In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/crypto-regulations-yirifi/mappingtoBERT_6.csv
/kaggle/input/crypto-regulations-yirifi/mappingtoBERT_5.csv
/kaggle/input/crypto-regulations-yirifi/output_improved.csv
/kaggle/input/crypto-regulations-yirifi/master_data_gpt4o_improved_latest.csv
/kaggle/input/crypto-regulations-yirifi/CSV_data_gpt_4o/CSV_data_gpt_4o/18.csv
/kaggle/input/crypto-regulations-yirifi/CSV_data_gpt_4o/CSV_data_gpt_4o/20.csv
/kaggle/input/crypto-regulations-yirifi/CSV_data_gpt_4o/CSV_data_gpt_4o/1.csv
/kaggle/input/crypto-regulations-yirifi/CSV_data_gpt_4o/CSV_data_gpt_4o/24.csv
/kaggle/input/crypto-regulations-yirifi/CSV_data_gpt_4o/CSV_data_gpt_4o/11.csv
/kaggle/input/crypto-regulations-yirifi/CSV_data_gpt_4o/CSV_data_gpt_4o/30.csv
/kaggle/input/crypto-regulations-yirifi/CSV_data_gpt_4o/CSV_data_gpt_4o/17.csv
/kaggle/input/crypto-regulations-yirifi/CSV_data_gpt_4o/CSV_data_gpt_4o/16.csv
/kaggle/input/crypto-regulations-yirifi/CSV_data_gpt_4o/CSV_data_gpt_4o/3.csv
/kaggle/input/crypto-regulations-

# Data import and cleaning up the data

In [2]:
df = pd.read_csv("/kaggle/input/crypto-regulations-yirifi/mappingtoBERT_6.csv")

In [3]:
df.shape

(5965, 4)

In [4]:
df.tail(10)

,Unnamed: 0,Content,Concepts,bert_concepts
5955,5955,Content|ÄúRegulationsÄù|means the Virtual As...,"amended, related activities, 2023, supplemente...","regulations, virtual assets"
5956,5956,Content|ÄúReserve AssetsÄù|has the meaning a...,"frva rules, reserve assets, rule i.a.5",rules
5957,5957,Content|ÄúSenior ManagementÄù|has the meanin...,"company rulebook, senior management",rulebook
5958,5958,Content|ÄúSignificant FRVA IssuerÄù|means a ...,"rule i.c.1, vara, frva rules, vasp, significan...","rules, vasp"
5959,5959,Content|ÄúTechnology and Information\nRuleboo...,"amended, information, vara, technology, regula...","regulations, rulebook"
5960,5960,Content|ÄúVA Issuance RulebookÄù|means the V...,"amended, supplemented, vara, virtual asset iss...","regulations, virtual assets, rulebook"
5961,5961,Content|ÄúVARAÄù|means the Dubai Virtual Ass...,"vara, dubai, regulatory authority, virtual assets","authority, virtual assets, regulator"
5962,5962,Content|ÄúVirtual AssetÄù or ÄúVAÄù|has th...,"dubai va law, va, regulatory, compliance, virt...","compliance, virtual assets, regulator"
5963,5963,Content|ÄúWhitepaperÄù|has the meaning ascri...,"bank, whitepaper, regulatory compliance, compl...","compliance, virtual assets, rulebook, regulator"
5964,5964,Content|ÄúWorking DayÄù|means any day which ...,"public holiday, weekend, bank, hong kong, regu...","compliance, virtual assets, regulator"


In [5]:
import re
# Function to check for URLs
def contains_url(text):
    text = str(text)
    return bool(re.search(r'http\S+|www\S+|https\S+', text))

# Function to check for user references (@user)
def contains_user_reference(text):
    text = str(text)
    return bool(re.search(r'\@\w+', text))

# Function to check for hashtags
def contains_hashtag(text):
    text = str(text)
    return bool(re.search(r'\#\w+', text))

# Function to check for non-alphanumeric characters
def contains_non_alphanumeric(text):
    text = str(text)
    return bool(re.search(r'[^a-zA-Z0-9\s]', text))

# Apply the functions to the DataFrame
df['contains_url'] = df['Content'].apply(contains_url)
df['contains_user_reference'] = df['Content'].apply(contains_user_reference)
df['contains_hashtag'] = df['Content'].apply(contains_hashtag)
df['contains_non_alphanumeric'] = df['Content'].apply(contains_non_alphanumeric)

# Check rows that match each condition
url_count = df['contains_url'].sum()
user_reference_count = df['contains_user_reference'].sum()
hashtag_count = df['contains_hashtag'].sum()
non_alphanumeric_count = df['contains_non_alphanumeric'].sum()

# Print results
print(f"Number of rows with URLs: {url_count}")
print(f"Number of rows with user references: {user_reference_count}")
print(f"Number of rows with hashtags: {hashtag_count}")
print(f"Number of rows with non-alphanumeric characters: {non_alphanumeric_count}")



Number of rows with URLs: 22
Number of rows with user references: 12
Number of rows with hashtags: 0
Number of rows with non-alphanumeric characters: 5913


In [6]:
df.tail(5)

,Unnamed: 0,Content,Concepts,bert_concepts,contains_url,contains_user_reference,contains_hashtag,contains_non_alphanumeric
5960,5960,Content|ÄúVA Issuance RulebookÄù|means the V...,"amended, supplemented, vara, virtual asset iss...","regulations, virtual assets, rulebook",False,False,False,True
5961,5961,Content|ÄúVARAÄù|means the Dubai Virtual Ass...,"vara, dubai, regulatory authority, virtual assets","authority, virtual assets, regulator",False,False,False,True
5962,5962,Content|ÄúVirtual AssetÄù or ÄúVAÄù|has th...,"dubai va law, va, regulatory, compliance, virt...","compliance, virtual assets, regulator",False,False,False,True
5963,5963,Content|ÄúWhitepaperÄù|has the meaning ascri...,"bank, whitepaper, regulatory compliance, compl...","compliance, virtual assets, rulebook, regulator",False,False,False,True
5964,5964,Content|ÄúWorking DayÄù|means any day which ...,"public holiday, weekend, bank, hong kong, regu...","compliance, virtual assets, regulator",False,False,False,True


In [7]:
df = df[["Content","bert_concepts"]]

In [8]:
import re
import string

def preprocess_text(text):
    text = str(text)    
    text = re.sub(r'<[^>]+>', '', text) # Remove HTML tags
    text = text.lower()
    text = re.sub(r'http\S+|www\S+|https\S+', '', text, flags=re.MULTILINE)# Remove URLs
    text = re.sub(r'\@\w+|\#','', text) # Remove user @ references and hashtags
    text = text.translate(str.maketrans('', '', string.punctuation)) # Remove punctuation    
    text = re.sub(r'[^a-zA-Z0-9\s]', ' ', text) # Remove non-alphanumeric character    
    text = re.sub(r'\s+', ' ', text).strip() # Remove extra whitespace(only single white space is left)
    
    return text

df['Content'] = df['Content'].apply(preprocess_text) # Applying the preprocess_text function


In [9]:
df.tail(5)

,Content,bert_concepts
5960,content va issuance rulebook means the virtual...,"regulations, virtual assets, rulebook"
5961,content vara means the dubai virtual assets re...,"authority, virtual assets, regulator"
5962,content virtual asset or va has the meaning as...,"compliance, virtual assets, regulator"
5963,content whitepaper has the meaning ascribed to...,"compliance, virtual assets, rulebook, regulator"
5964,content working day means any day which is not...,"compliance, virtual assets, regulator"


# taking out 10 random samples for analysis

In [10]:
testing_df = df.sample(n=10, random_state=42)
df = df.drop(testing_df.index)

In [11]:
df.shape

(5955, 2)

In [12]:
missing_values_count = df['bert_concepts'].isnull().sum()
print("Number of rows with NaN in bert_concepts:", missing_values_count)

Number of rows with NaN in bert_concepts: 579


## Remove rows with no bert_concepts

In [13]:
df = df.dropna(subset=['bert_concepts'])

In [14]:
df.shape

(5376, 2)

# Converting to Bert Data Format

In [15]:
import pandas as pd
import re
from sklearn.model_selection import train_test_split
from datasets import Dataset, DatasetDict

In [16]:
df.columns

Index(['Content', 'bert_concepts'], dtype='object')

In [17]:

df['bert_concepts'] = df['bert_concepts'].apply(lambda x: x.split(', ')) # make list of words
all_concepts = set(concept for row in df['bert_concepts'] for concept in row) # Create a list of all unique concepts


In [18]:
#all_concepts

In [19]:
# One-hot encode each concept for each row
one_hot_encoded_rows = []
for index, row in df.iterrows():
    one_hot_row = {'Content': row['Content']}
    for concept in all_concepts:
        one_hot_row[concept] = 1 if concept in row['bert_concepts'] else 0
    one_hot_encoded_rows.append(one_hot_row)


In [20]:
#one_hot_encoded_rows[:2]

In [21]:
df = pd.DataFrame(one_hot_encoded_rows)

In [22]:
df.head(3)

,Content,policy,investor protection,rulebook,circular,oversight,guideline,conduct,regulations,client,...,investor,sanctions,transactions,travel rule,screening,risks,monitoring,safeguard,cft,policies
0,the new licensing regime for centralised virtu...,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
1,the sfc is issuing this circular to highlight ...,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,the sfc has published the following guidelines...,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0


# Class/Label Distribution

In [23]:
import pandas as pd
l1 = list(df.columns)[1:]
summary_list = []
for i in l1:
    label_counts = df[i].value_counts()
    summary_list.append({
        'Label': i,
        '0s': label_counts.get(0, 0),
        '1s': label_counts.get(1, 0)
    })


summary_df = pd.DataFrame(summary_list)
summary_df_sorted = summary_df.sort_values(by='1s', ascending=False)
summary_df_sorted


,Label,0s,1s
32,virtual assets,3149,2227
34,compliance,3778,1598
24,regulator,3798,1578
8,client,4090,1286
50,risks,4555,821
18,aml,4650,726
37,vasp,4727,649
16,license,4741,635
25,operator,4787,589
36,requirements,4834,542


# Apply oversmapling of Minorty classes


In [24]:
!pip install git+https://github.com/phiyodr/multilabel-oversampling

  Cloning https://github.com/phiyodr/multilabel-oversampling to /tmp/pip-req-build-s5d2lx3t
  Running command git clone --filter=blob:none --quiet https://github.com/phiyodr/multilabel-oversampling /tmp/pip-req-build-s5d2lx3t
  Resolved https://github.com/phiyodr/multilabel-oversampling to commit 5fa066171b8df7ef369fe4f0c674f9576a106b0e
  Preparing metadata (setup.py) ... - done
  Created wheel for multilabel-oversampling: filename=multilabel_oversampling-0.1.3-py3-none-any.whl size=6225 sha256=f9a92334a209c055c984feab6755b15ed1ac564c3b079822d0906d77c0c604da
  Stored in directory: /tmp/pip-ephem-wheel-cache-5af8bahi/wheels/2a/ba/2b/c3f093556ca8b37e8714e4748090daa11b45128ccb614665de
Successfully built multilabel-oversampling


In [25]:
import multilabel_oversampling as mo

mo.seed_everything(20)
df = df # difficult fake dataset with very high dependency of y1 and y2
ml_oversampler = mo.MultilabelOversampler(number_of_adds = 1000, number_of_tries=200,details=False, plot=False)
l1 = list(df.columns)[1:]
df_new = ml_oversampler.fit(df,target_list=l1)
#>Start the upsampling process.
#>Iteration:  11%|████████████████                                        | 11/100 [00:00<00:01, 48.43it/s]
#>Iter 11: No improvement after 100 tries.
#>Sampling done.
#>
#>Dataset size original: 20; Upsampled dataset size: 31
#>Original target distribution:  {'y1': 16, 'y2': 12, 'y3': 4, 'y4': 4}
#>Upsampled target distribution: {'y1': 19, 'y2': 12, 'y3': 15, 'y4': 15}

#ml_oversampler.plot_all_tries()

Start the upsampling process.


Iteration: 100%|██████████| 1000/1000 [00:32<00:00, 31.08it/s]


In [26]:
#ml_oversampler.plot_results()

In [27]:
#change df to oversampled one
df = df_new

In [28]:
# from sklearn.feature_extraction.text import TfidfVectorizer
# from imblearn.over_sampling import SMOTE
# from sklearn.multiclass import OneVsRestClassifier
# from sklearn.linear_model import LogisticRegression
# from sklearn.metrics import classification_report

# df = df_new[0]

# X = df['Content']
# y = df.drop(columns=['Content'])

# # Split the data into training and testing sets
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


# vectorizer = TfidfVectorizer(max_features=5000)
# X_train_vec = vectorizer.fit_transform(X_train)
# X_test_vec = vectorizer.transform(X_test)




In [29]:
# from sklearn.multiclass import OneVsRestClassifier
# from sklearn.linear_model import LogisticRegression
# from sklearn.metrics import classification_report

# # Train the classifier
# classifier = OneVsRestClassifier(LogisticRegression(solver='liblinear'))
# classifier.fit(X_train_vec, y_train)

# # Predict on the test set
# y_pred = classifier.predict(X_test_vec)

# # Evaluate the classifier
# print(classification_report(y_test, y_pred, target_names=y.columns))


In [30]:
import pandas as pd
from datasets import Dataset

In [31]:
#Splitting DataFrame into train, test, and validation sets
train_df, test_valid_df = train_test_split(df, test_size=0.3, random_state=42)
test_df, validation_df = train_test_split(test_valid_df, test_size=0.33, random_state=42)

# Convert DataFrames to Hugging Face Datasets
train_dataset = Dataset.from_pandas(train_df)
test_dataset = Dataset.from_pandas(test_df)
validation_dataset = Dataset.from_pandas(validation_df)

# Create DatasetDict
dataset = DatasetDict({
    "train": train_dataset,
    "test": test_dataset,
    "validation": validation_dataset
})




In [32]:
dataset

DatasetDict({
    train: Dataset({
        features: ['Content', 'policy', 'investor protection', 'rulebook', 'circular', 'oversight', 'guideline', 'conduct', 'regulations', 'client', 'cdd', 'identity', 'due diligence', 'code of conduct', 'risk assessment', 'pep', 'risk management', 'license', 'rules', 'aml', 'reporting', 'beneficiary information', 'vasps', 'audit', 'transaction', 'regulator', 'operator', 'authority', 'controls', 'kyc', 'record keeping', 'criminal', 'transfer', 'virtual assets', 'disclosure', 'compliance', 'legislation', 'requirements', 'vasp', 'dlt', 'entities', 'legal', 'person', 'insider dealing', 'suspicious transactions', 'anti-bribery', 'investor', 'sanctions', 'transactions', 'travel rule', 'screening', 'risks', 'monitoring', 'safeguard', 'cft', 'policies', '__index_level_0__'],
        num_rows: 4463
    })
    test: Dataset({
        features: ['Content', 'policy', 'investor protection', 'rulebook', 'circular', 'oversight', 'guideline', 'conduct', 'regulations

In [33]:
[i for i in dataset['train']][:2]

[{'Content': 'c the amount of interest accrued from and credited to the client s account for lending transactions both total and during the reporting period and',
  'policy': 0,
  'investor protection': 0,
  'rulebook': 0,
  'circular': 0,
  'oversight': 0,
  'guideline': 0,
  'conduct': 0,
  'regulations': 0,
  'client': 1,
  'cdd': 0,
  'identity': 0,
  'due diligence': 0,
  'code of conduct': 0,
  'risk assessment': 0,
  'pep': 0,
  'risk management': 0,
  'license': 0,
  'rules': 0,
  'aml': 0,
  'reporting': 1,
  'beneficiary information': 0,
  'vasps': 0,
  'audit': 0,
  'transaction': 1,
  'regulator': 0,
  'operator': 0,
  'authority': 0,
  'controls': 0,
  'kyc': 0,
  'record keeping': 0,
  'criminal': 0,
  'transfer': 0,
  'virtual assets': 0,
  'disclosure': 0,
  'compliance': 0,
  'legislation': 0,
  'requirements': 0,
  'vasp': 0,
  'dlt': 0,
  'entities': 0,
  'legal': 0,
  'person': 0,
  'insider dealing': 0,
  'suspicious transactions': 0,
  'anti-bribery': 0,
  'invest

# Data Ready Now Training

In [34]:
!pip install -q transformers datasets

ERROR: Could not install packages due to an OSError: [Errno 2] No such file or directory: '/opt/conda/lib/python3.10/site-packages/aiohttp-3.9.1.dist-info/METADATA'



Add a linear layer on top of the base model, which is used to produce a tensor of shape (batch_size, num_labels), indicating the unnormalized scores for a number of labels for every example in the batch.

### List containing all the labels

In [35]:
labels = [label for label in dataset['train'].features.keys() if label not in ['Content','__index_level_0__']]
#Create 2 dictionaries that map labels to integers and back.
id2label = {idx:label for idx, label in enumerate(labels)}
label2id = {label:idx for idx, label in enumerate(labels)}
labels

['policy',
 'investor protection',
 'rulebook',
 'circular',
 'oversight',
 'guideline',
 'conduct',
 'regulations',
 'client',
 'cdd',
 'identity',
 'due diligence',
 'code of conduct',
 'risk assessment',
 'pep',
 'risk management',
 'license',
 'rules',
 'aml',
 'reporting',
 'beneficiary information',
 'vasps',
 'audit',
 'transaction',
 'regulator',
 'operator',
 'authority',
 'controls',
 'kyc',
 'record keeping',
 'criminal',
 'transfer',
 'virtual assets',
 'disclosure',
 'compliance',
 'legislation',
 'requirements',
 'vasp',
 'dlt',
 'entities',
 'legal',
 'person',
 'insider dealing',
 'suspicious transactions',
 'anti-bribery',
 'investor',
 'sanctions',
 'transactions',
 'travel rule',
 'screening',
 'risks',
 'monitoring',
 'safeguard',
 'cft',
 'policies']

In [36]:
id2label

{0: 'policy',
 1: 'investor protection',
 2: 'rulebook',
 3: 'circular',
 4: 'oversight',
 5: 'guideline',
 6: 'conduct',
 7: 'regulations',
 8: 'client',
 9: 'cdd',
 10: 'identity',
 11: 'due diligence',
 12: 'code of conduct',
 13: 'risk assessment',
 14: 'pep',
 15: 'risk management',
 16: 'license',
 17: 'rules',
 18: 'aml',
 19: 'reporting',
 20: 'beneficiary information',
 21: 'vasps',
 22: 'audit',
 23: 'transaction',
 24: 'regulator',
 25: 'operator',
 26: 'authority',
 27: 'controls',
 28: 'kyc',
 29: 'record keeping',
 30: 'criminal',
 31: 'transfer',
 32: 'virtual assets',
 33: 'disclosure',
 34: 'compliance',
 35: 'legislation',
 36: 'requirements',
 37: 'vasp',
 38: 'dlt',
 39: 'entities',
 40: 'legal',
 41: 'person',
 42: 'insider dealing',
 43: 'suspicious transactions',
 44: 'anti-bribery',
 45: 'investor',
 46: 'sanctions',
 47: 'transactions',
 48: 'travel rule',
 49: 'screening',
 50: 'risks',
 51: 'monitoring',
 52: 'safeguard',
 53: 'cft',
 54: 'policies'}

# Tokenization

### As models like BERT don't expect text as direct input, but rather input_ids, etc., we tokenize the text using the tokenizer. We will use AutoTokenizer API, which will automatically load the appropriate tokenizer based on the checkpoint on the hub.

### What's a bit tricky is that we also need to provide labels to the model. For multi-label text classification, this is a matrix of shape (batch_size, num_labels). Also important: this should be a tensor of floats rather than integers, otherwise PyTorch' BCEWithLogitsLoss (which the model will use) will complain

In [37]:
from transformers import AutoTokenizer
import numpy as np

tokenizer = AutoTokenizer.from_pretrained("ProsusAI/finbert")

def preprocess_data(examples, max_length=128):
  # take a batch of texts
  text = examples['Content']
  batch_size = len(text)
  # encode them
  encoding = tokenizer(text, padding="max_length", truncation=True, max_length=max_length)
  # add labels
  labels_batch = {k: examples[k] for k in examples.keys() if k in labels}
  # create numpy array of shape (batch_size, num_labels)
  labels_matrix = np.zeros((len(text), len(labels)))
  # fill numpy array
  for idx, label in enumerate(labels):
    labels_matrix[:, idx] = labels_batch[label]

  encoding["labels"] = labels_matrix.tolist()
  
  return encoding

tokenizer_config.json:   0%|          | 0.00/252 [00:00<?, ?B/s]

/opt/conda/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/758 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

In [38]:
encoded_dataset = dataset.map(preprocess_data, batched=True, remove_columns=dataset['train'].column_names)

Map:   0%|          | 0/4463 [00:00<?, ? examples/s]

Map:   0%|          | 0/1281 [00:00<?, ? examples/s]

Map:   0%|          | 0/632 [00:00<?, ? examples/s]

In [39]:
encoded_dataset

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask', 'labels'],
        num_rows: 4463
    })
    test: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask', 'labels'],
        num_rows: 1281
    })
    validation: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask', 'labels'],
        num_rows: 632
    })
})

In [40]:
encoded_dataset['train']

Dataset({
    features: ['input_ids', 'token_type_ids', 'attention_mask', 'labels'],
    num_rows: 4463
})

In [41]:
example = encoded_dataset['train'][0]
tokenizer.decode(example['input_ids'])

2024-06-05 18:35:37.907181: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-06-05 18:35:37.907282: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-06-05 18:35:38.027075: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


'[CLS] c the amount of interest accrued from and credited to the client s account for lending transactions both total and during the reporting period and [SEP] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD]'

In [42]:
#example['labels']

In [43]:
#use id to label dictionary
[id2label[idx] for idx, label in enumerate(example['labels']) if label == 1.0]
encoded_dataset.set_format("torch") 

### Here we define a model that includes a pre-trained base (i.e. the weights from bert-base-uncased) are loaded, with a random initialized classification head (linear layer) on top. One should fine-tune this head, together with the pre-trained base on a labeled dataset.

### We set the problem_type to be "multi_label_classification", as this will make sure the appropriate loss function is used (namely BCEWithLogitsLoss). We also make sure the output layer has len(labels) output neurons, and we set the id2label and label2id mappings

In [44]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained("ProsusAI/finbert", 
                                                           problem_type="multi_label_classification", 
                                                           num_labels=len(labels),
                                                           id2label=id2label,
                                                           label2id=label2id,
                                                          ignore_mismatched_sizes=True )

pytorch_model.bin:   0%|          | 0.00/438M [00:00<?, ?B/s]

/opt/conda/lib/python3.10/site-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at ProsusAI/finbert and are newly initialized because the shapes did not match:
- classifier.weight: found shape torch.Size([3, 768]) in the checkpoint and torch.Size([55, 768]) in the model instantiated
- classifier.bias: found shape torch.Size([3]) in the checkpoint and torch.Size([55]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [45]:
batch_size = 16
metric_name = "f1"

In [46]:
from transformers import TrainingArguments, Trainer

args = TrainingArguments(
    f"bert-finetuned-sem_eval-english",
    evaluation_strategy = "epoch",
    save_strategy = "epoch",
    learning_rate=1e-4,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=15,
    weight_decay=0.01,
    load_best_model_at_end=True,
    metric_for_best_model=metric_name
)

/opt/conda/lib/python3.10/site-packages/transformers/training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


we need to define a compute_metrics function, that returns a dictionary with the desired metric values.

source: https://jesusleal.io/2021/04/21/Longformer-multilabel-classification/

# 1. F1 Score
The F1 score is the harmonic mean of precision and recall. It provides a single metric that balances both the precision and recall, making it useful for evaluating models where you need to balance the two, especially in the presence of imbalanced classes.

Precision: The ratio of true positive predictions to the total number of positive predictions (true positives + false positives). It measures the accuracy of the positive predictions.
Recall: The ratio of true positive predictions to the total number of actual positives (true positives + false negatives). It measures the ability to capture all relevant instances.
The formula for the F1 score is:
F1 score
=
2
×
(
Precision
×
Recall
Precision
+
Recall
)
F1 score=2×( 
Precision+Recall
Precision×Recall
​
 )

# 2. ROC AUC Score
The ROC AUC (Receiver Operating Characteristic Area Under Curve) score is a metric used to evaluate the performance of a binary classifier. It measures the ability of the classifier to distinguish between classes.

ROC Curve: A plot of the true positive rate (recall) against the false positive rate (1-specificity) at various threshold settings.
AUC: The area under the ROC curve. A higher AUC indicates better performance, with a score of 1 representing a perfect classifier and a score of 0.5 representing a random classifier.

# 3. Accuracy Score
Accuracy is the ratio of correctly predicted instances to the total instances. It is the most intuitive performance measure, but it can be misleading when dealing with imbalanced datasets.

The formula for accuracy is:
Accuracy
=
Number of Correct Predictions
Total Number of Predictions
Accuracy= 
Total Number of Predictions
Number of Correct Predictions



In [47]:
from sklearn.metrics import f1_score, roc_auc_score, accuracy_score
from transformers import EvalPrediction
import torch
    

def multi_label_metrics(predictions, labels, threshold=0.5):
    # first, apply sigmoid on predictions which are of shape (batch_size, num_labels)
    sigmoid = torch.nn.Sigmoid()
    probs = sigmoid(torch.Tensor(predictions))
    # next, use threshold to turn them into integer predictions
    y_pred = np.zeros(probs.shape)
    y_pred[np.where(probs >= threshold)] = 1
    # finally, compute metrics
    y_true = labels
    f1_micro_average = f1_score(y_true=y_true, y_pred=y_pred, average='micro')
    roc_auc = roc_auc_score(y_true, y_pred, average = 'micro')
    accuracy = accuracy_score(y_true, y_pred)
    # return as dictionary
    metrics = {'f1': f1_micro_average,
               'roc_auc': roc_auc,
               'accuracy': accuracy}
    return metrics

def compute_metrics(p: EvalPrediction):
    preds = p.predictions[0] if isinstance(p.predictions, tuple) else p.predictions
    result = multi_label_metrics(predictions=preds,labels=p.label_ids)
    return result
  
   


In [48]:
encoded_dataset['train'][0]['labels'].type()

'torch.FloatTensor'

In [49]:
encoded_dataset['train']['input_ids'][0]

tensor([  101,  1039,  1996,  3815,  1997,  3037, 16222, 28551,  2013,  1998,
         5827,  2000,  1996,  7396,  1055,  4070,  2005, 18435, 11817,  2119,
         2561,  1998,  2076,  1996,  7316,  2558,  1998,   102,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0])

In [50]:
# #forward pass
# #, attention_mask=attention_mask
# outputs = model(input_ids=encoded_dataset['train']['input_ids'][0].unsqueeze(0), labels=encoded_dataset['train'][0]['labels'].unsqueeze(0))
# outputs

In [51]:
from transformers import AutoModelForSequenceClassification, Trainer

In [52]:

from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()
secret_value_0 = user_secrets.get_secret("WANDB_API_KEY")


In [53]:
import os
os.environ['WANDB_API_KEY'] ="313a57558bcaee784e68d1654f7915a0b463a341"

In [54]:
trainer = Trainer(
    model,
    args,
    train_dataset=encoded_dataset["train"],
    eval_dataset=encoded_dataset["validation"],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,

)

In [55]:
trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Currently logged in as: anshtanwar007 (anshta). Use `wandb login --relogin` to force relogin
wandb: Tracking run with wandb version 0.17.0
wandb: Run data is saved locally in /kaggle/working/wandb/run-20240605_183556-lq50x67t
wandb: Run `wandb offline` to turn off syncing.
wandb: Syncing run bert-finetuned-sem_eval-english
wandb: ⭐️ View project at https://wandb.ai/anshta/huggingface
wandb: 🚀 View run at https://wandb.ai/anshta/huggingface/runs/lq50x67t


Epoch,Training Loss,Validation Loss,F1,Roc Auc,Accuracy
1,No log,0.152169,0.307380,0.596594,0.014241
2,0.173000,0.104969,0.609391,0.728175,0.204114
3,0.173000,0.076729,0.749615,0.817064,0.368671
4,0.084600,0.065295,0.814751,0.868277,0.501582
5,0.084600,0.059849,0.854007,0.905017,0.588608
6,0.049500,0.055630,0.834768,0.890976,0.526899
7,0.049500,0.051750,0.865844,0.911102,0.609177
8,0.032000,0.048912,0.880585,0.923681,0.653481
9,0.021700,0.049871,0.878115,0.922310,0.651899
10,0.021700,0.050741,0.879026,0.923096,0.658228


TrainOutput(global_step=4185, training_loss=0.04827052176782022, metrics={'train_runtime': 982.4923, 'train_samples_per_second': 68.138, 'train_steps_per_second': 4.26, 'total_flos': 4405587870885120.0, 'train_loss': 0.04827052176782022, 'epoch': 15.0})

In [56]:
trainer.evaluate()

{'eval_loss': 0.050228655338287354,
 'eval_f1': 0.8853347790729195,
 'eval_roc_auc': 0.9256824839997744,
 'eval_accuracy': 0.6740506329113924,
 'eval_runtime': 2.4972,
 'eval_samples_per_second': 253.086,
 'eval_steps_per_second': 16.018,
 'epoch': 15.0}

# Inference
add Codeadd Markdown
The logits that come out of the model are of shape (batch_size, num_labels). As we are only forwarding a single sentence through the model, the batch_size equals 1.

The logits is a tensor that contains the (unnormalized) scores for every individual label.

The logits that come out of the model are of shape (batch_size, num_labels). As we are only forwarding a single sentence through the model, the batch_size equals 1.

The logits is a tensor that contains the (unnormalized) scores for every individual label.

In [57]:
def get_answer(text):
    
    encoding = tokenizer(text, return_tensors="pt")
    encoding = {k: v.to(trainer.model.device) for k,v in encoding.items()}
    outputs = trainer.model(**encoding)
    logits = outputs.logits
    #print(logits.shape)
    
    # apply sigmoid + threshold
    sigmoid = torch.nn.Sigmoid()
    probs = sigmoid(logits.squeeze().cpu())
    #print(probs)
    predictions = np.zeros(probs.shape)
    predictions[np.where(probs >= 0.3)] = 1
    #print(predictions)
    #predicted_labels = [id2label[idx] for idx, label in enumerate(predictions) if label == 1.0]
    predicted_labels = {id2label[idx]: prob.item() for idx, prob in enumerate(probs) if predictions[idx] == 1}
    
    print()
    return predicted_labels


In [58]:
text = "The new licensing regime for centralised virtual asset trading platforms under the Anti-MoneyLaundering and Counter-Terrorist Financing Ordinance (Cap. 615) (AMLO) will come intoeffect on 1 June 2023. Under the new regime, centralised virtual asset trading platformsoperating in Hong Kong will need to apply to the Securities and Futures Commission (SFC)for a licence under the Securities and Futures Ordinance (Cap 571) (SFO) and/or the AMLO(Dual Licence Arrangement)"
get_answer(text)

{'license': 0.9844382405281067,
 'aml': 0.988938570022583,
 'regulator': 0.5525399446487427,
 'virtual assets': 0.9842027425765991,
 'compliance': 0.4846854507923126,
 'cft': 0.9842115044593811}

In [59]:
testing_df.shape

(10, 2)

In [60]:
for index, row in testing_df.iterrows():

    text = row['Content']
    predictions = get_answer(text)
    actual_labels = testing_df.columns[1:][row[1:] == 1].tolist()
    
    print(text,'\n')    
    print(f"Predicted Labels: {predictions}")
    print(f"Actual Labels: {actual_labels}")
    print("________________________________________________________________________________")


for the purpose of matters reportable by auditors under sections 53zsd4ai and 53zsd4bi of the amlo such matters are 

Predicted Labels: {'aml': 0.9713087677955627, 'reporting': 0.740779459476471, 'audit': 0.8654091954231262}
Actual Labels: []
________________________________________________________________________________

it may be used for storing the value of an amount of money that i is paid into the facility from time to time and ii may be stored on the facility under the rules of the facility and 

Predicted Labels: {'rules': 0.9895932674407959}
Actual Labels: []
________________________________________________________________________________

while an fi usually can identify who the beneficial owner of a customer is in the course of understanding the ownership and control structure of the customer the fi may obtain an undertaking or declarationfn28 from the customer on the identity of and the information relating to its beneficial owner when identifying a beneficial owner the f

# Probability

In [61]:
# outputs = trainer.model(**encoding)
# logits = outputs.logits
# logits.shape

The logits that come out of the model are of shape (batch_size, num_labels). As we are only forwarding a single sentence through the model, the batch_size equals 1.

The logits is a tensor that contains the (unnormalized) scores for every individual label

To turn them into actual predicted labels, we first apply a sigmoid function independently to every score, such that every score is turned into a number between 0 and 1, that can be interpreted as a "probability" for how certain the model is that a given class belongs to the input text.

Next, we use a threshold (typically, 0.5) to turn every probability into either a 1 (which means, we predict the label for the given example) or a 0 (which means, we don't predict the label for the given example)

In [62]:
# # apply sigmoid + threshold
# sigmoid = torch.nn.Sigmoid()
# probs = sigmoid(logits.squeeze().cpu())
# predictions = np.zeros(probs.shape)
# predictions[np.where(probs >= 0.5)] = 1
# # turn predicted id's into actual label names
# predicted_labels = [id2label[idx] for idx, label in enumerate(predictions) if label == 1.0]
# print(predicted_labels)